## Contextvars

contextvars are kinda like thread-local but more granular.

In [1]:
import contextvars

define a new context var:

In [2]:
request_id = contextvars.ContextVar('request_id')

we can get var value with `.get()`

In [3]:
request_id.get()

LookupError: <ContextVar name='request_id' at 0x10629c0e0>

Actually we can't, because request_id is unbound nad has no default value

In [4]:
request_id.set('abc')

<Token var=<ContextVar name='request_id' at 0x10629c0e0> at 0x106766f40>

Now `.get()` works

In [5]:
request_id.get()

'abc'

We can create a new context and run some code in this new context

In [6]:
def handle_request(new_request_id):
    print(f"before {request_id.get()=}")
    request_id.set(new_request_id)
    print(f"after {request_id.get()=}")

In [7]:
ctx = contextvars.copy_context()
ctx.run(handle_request, 'def')

before request_id.get()='abc'
after request_id.get()='def'


Value of request_id didn't change in the primary context:

In [8]:
request_id.get()

'abc'

Under the hood, there's a thread-local variable, that is a mapping of contextvars.
You can copy it in O(1) with `copy_context`.
`X.run()` reassigns this thread-local to point to `X` for a duration of `run`. It restores the previous context at exit.

Also `contextvars` are asyncio-friendly, as `asyncio.create_task`, `asyncio.call_*` take an additional context argument. So you can have a single context for your task.

So if you just create a new thread in your `asyncio` then you'll have no current context in a thread. You need to be careful to run your code in a thread by using `context.run` or by using context-aware `asyncio.to_thread`.